In [1]:
# imports
# from transformers import pipeline
from helpers import Video_loader, Image_loader, Captioner, Audio_transcriber, Fusion_helper
import tensorflow as tf

In [2]:
VIDEOS_DIRECTORY = "videos"
FRAME_FREQUENCY = 300

In [3]:
# video captions section
video_loader = Video_loader(VIDEOS_DIRECTORY, FRAME_FREQUENCY)

captions_dict = video_loader.process()
# need to tune this up but for now it's whatever - we have frame visual captions now

print("------------------------------------------------------------------------------------------\n END OF HELPERS \n")
print(captions_dict)

Init video loader
Make frames
Storing in videos\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_images
Start image loading
Init image loader
Init captioner


All model checkpoint layers were used when initializing TFBlipForConditionalGeneration.

All the layers of TFBlipForConditionalGeneration were initialized from the model checkpoint at Salesforce/blip-image-captioning-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlipForConditionalGeneration for predictions without further training.


Start image get captions


c:\Users\david\anaconda3\envs\research-env\lib\site-packages\transformers\generation\tf_utils.py:854: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


------------------------------------------------------------------------------------------
 END OF HELPERS 

{'videos\\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_images': [[{'generated_text': "the logo for los's employee training"}], [{'generated_text': 'a cartoon depicting a man cleaning a room'}], [{'generated_text': 'a man in a yellow shirt and tie is pointing at a computer screen'}], [{'generated_text': 'a man and woman are in a room with a tv screen'}], [{'generated_text': 'a man with glasses and a thought bubble above him'}], [{'generated_text': 'a man in a tie and shirt standing in a restaurant'}], [{'generated_text': 'a man with a mustache and a blue sky in the background'}], [{'generated_text': 'a man in a tie standing in front of a sign'}], [{'generated_text': 'a man with a beard and a blue sky in the background'}], [{'generated_text': 'a man in a tie and glasses is making a funny face'}], [{'generated_text': 'a man in a tie and shirt standing

In [4]:
# audio captions section
audio_transcriber = Audio_transcriber(VIDEOS_DIRECTORY)
audio_captions_dict = audio_transcriber.transcribe_video_folder(VIDEOS_DIRECTORY)

Transcribing video folder: videos
Filenames: 
[<DirEntry 'Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4'>]
Make folders and save audio
Storing in videos\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_audio
Transcribing video <DirEntry 'Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4'>
Audio split!
videos\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_audio\chunk1.wav : hello and welcome to the Los Pollos Hermanos family my name is Gustavo but you can call me I am thrilled that you'll be joining our team each and every day we serve our customers exceptional we take pride everything that we do and after this 10-week online seminar I am confident that you would write it I like to think I see things and people to begin I'd like to talk about the cornerstone of the Los Pollos Hermanos brand communication as an employee have lost boy from are you set the tone for the entire din

In [5]:
# explore dicts
print("---FRAME CAPTIONS---")
print(captions_dict)
print("\n\n---AUDIO CAPTIONS---")
print(audio_captions_dict)

---FRAME CAPTIONS---
{'videos\\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_images': [[{'generated_text': "the logo for los's employee training"}], [{'generated_text': 'a cartoon depicting a man cleaning a room'}], [{'generated_text': 'a man in a yellow shirt and tie is pointing at a computer screen'}], [{'generated_text': 'a man and woman are in a room with a tv screen'}], [{'generated_text': 'a man with glasses and a thought bubble above him'}], [{'generated_text': 'a man in a tie and shirt standing in a restaurant'}], [{'generated_text': 'a man with a mustache and a blue sky in the background'}], [{'generated_text': 'a man in a tie standing in front of a sign'}], [{'generated_text': 'a man with a beard and a blue sky in the background'}], [{'generated_text': 'a man in a tie and glasses is making a funny face'}], [{'generated_text': 'a man in a tie and shirt standing in a room'}]]}


---AUDIO CAPTIONS---
{<DirEntry 'Los Pollos Hermanos Employee Training

In [6]:
# generate new frame caption dict with cleaned data
video_captions_dict = {}
for video_name in captions_dict.keys():
    combined_text = ""
    for top_level in captions_dict[video_name]:
        inside_dict = top_level[0]
        combined_text = combined_text + ' ' + inside_dict["generated_text"]
    video_captions_dict[video_name] = combined_text

print(video_captions_dict)

{'videos\\Los Pollos Hermanos Employee Training_ Communication _ Better Call Saul.mp4_images': " the logo for los's employee training a cartoon depicting a man cleaning a room a man in a yellow shirt and tie is pointing at a computer screen a man and woman are in a room with a tv screen a man with glasses and a thought bubble above him a man in a tie and shirt standing in a restaurant a man with a mustache and a blue sky in the background a man in a tie standing in front of a sign a man with a beard and a blue sky in the background a man in a tie and glasses is making a funny face a man in a tie and shirt standing in a room"}


In [7]:
# fusion section
# steps to achieve:
# - combine string per video, do word embeddings with GloVe - theoretically, helper function here is complete!
# - build fusion layer - must see how to integrate the embeddings now
fusion_helper = Fusion_helper()
all_dicts = [video_captions_dict, audio_captions_dict]
final_vocab = fusion_helper.get_full_vocabulary(all_dicts)
embedding_layer = fusion_helper.build_embedding_layer_from_wordlist(final_vocab)
# TODO examine whether vectorisation and embedding layers are crosscompatible
vectorise_layer = fusion_helper.build_vectorise_layer_from_wordlist(final_vocab)
# yay - embedding layer made! going to also need a text vectorisation layer though that turns strings into vocab indices

Getting full vocabulary
Building embedding layer from wordlist
Getting embeddings from wordlist
Building embedding layer
Building vectorise layer


In [8]:
# model itself:
model = tf.keras.Sequential([
    vectorise_layer,
    embedding_layer,
    tf.keras.layers.Flatten(),
    tf.keras.layers.LSTM(units=1),
    tf.keras.layers.Dense(1)
])

In [9]:
# model compilation
# steps to achieve:
# - implement transformer model to analyse all text
# - decide on loss - binary cross-entropy most likely
# - decide on optimiser
# - compile model
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']
)

In [10]:
# training
# steps to achieve:
# - download all data
# - run preprocessing on all of the videos


In [11]:
# testing